In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 123074 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [3]:
!pip install dtcwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dtcwt: filename=dtcwt-0.12.0-py3-none-any.whl size=87869 sha256=4d3d934ffdc1634bae6b8e8058238872d20f4697a0e011134f6763081a5970ce
  Stored in directory: /root/.cache/pip/wheels/af/50/50/cf98b0e08812889d0f5789d271acbdab7ac7c3086c323a8e0a
Successfully built dtcwt


### Try to load the pre-trained model

In [4]:
import os
import re
import pickle
import numpy as np
SETS = ['training', 'validation', 'tests']
GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

### Load the dataset

In [5]:
import librosa
import numpy as np
from collections import Counter
import scipy
import dtcwt

trans = dtcwt.Transform1d(biort='antonini', qshift='qshift_d')

In [6]:
def calculate_entropy(list_values):
	counter_values = Counter(list_values).most_common()
	probabilities = [elem[1]/len(list_values) for elem in counter_values]
	entropy=scipy.stats.entropy(probabilities)
	return entropy

def calculate_statistics(list_values):
	n5 = np.nanpercentile(list_values, 5)
	n25 = np.nanpercentile(list_values, 25)
	n75 = np.nanpercentile(list_values, 75)
	n95 = np.nanpercentile(list_values, 95)
	median = np.nanpercentile(list_values, 50)
	mean = np.nanmean(list_values)
	std = np.nanstd(list_values)
	var = np.nanvar(list_values)
	rms = np.nanmean(np.sqrt(list_values**2))
	return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
	zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
	no_zero_crossings = len(zero_crossing_indices)
	mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
	no_mean_crossings = len(mean_crossing_indices)
	return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
	entropy = calculate_entropy(list_values)
	crossings = calculate_crossings(list_values)
	statistics = calculate_statistics(list_values)
	return [entropy] + crossings + statistics

## Given an audio file returns the corresponding wavelet features

In [7]:
def extract_dtcwt(file_path:str):
    d, fs = librosa.load(file_path)
    forw = trans.forward(d, nlevels=17)
    features = []
    for coeff in forw.highpasses:
        temp = (np.abs(coeff.squeeze()))
        features += get_features(temp)

    features += get_features(forw.lowpass.squeeze())
    return features

In [9]:
import numpy as np
import os

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    raise Exception(f"Not found the {genre}")

def get_data_from_path(data_path,genres, decoder):
    data = {'in': [], 'out': []}
    for genre in genres:
        files = os.listdir(data_path + genre)
        for filename in files:
            filepath = data_path + genre + '/' + filename
            data['in'].append(decoder(filepath))
            data['out'].append(index_genre(genre, genres))

    data = {'in': np.array(data['in']),'out': np.array(data['out'])}

    return data


In [ ]:
data_path_train = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'
data_path_test = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'
data_path_validation = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'


def get_data_train_wavelet():
    return get_data_from_path(data_path_train, genres, extract_dtcwt)
def get_data_test_wavelet():
    return get_data_from_path(data_path_test, genres, extract_dtcwt)
def get_data_validation_wavelet():
    return get_data_from_path(data_path_test, genres, extract_dtcwt)


Get a random scaled, train-test split

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
def encoder_scaler(X_dtcwt, y):
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)

    scaler = StandardScaler()
    X_dtcwt_scaled = scaler.fit_transform(X_dtcwt)
    X_dtcwt_train_f, X_dtcwt_test_f, y_dtcwt_train_f, y_dtcwt_test_f = train_test_split(X_dtcwt_scaled, y, test_size = 0.2,random_state=42)
    return X_dtcwt_train_f, X_dtcwt_test_f, y_dtcwt_train_f, y_dtcwt_test_f

In [11]:
dtcwt_rf = RandomForestClassifier(n_estimators=100,max_depth=13,bootstrap=False,random_state=42)

data_train = get_data_train_wavelet()
data_test = get_data_test_wavelet()
X_dtcwt_train_f = data_train ['in']
y_dtcwt_train_f = data_train ['out']
X_dtcwt_test_f =  data_test['in']
y_dtcwt_test_f =  data_test['out']

dtcwt_rf.fit(X_dtcwt_train_f,y_dtcwt_train_f)

with open('/content/drive/MyDrive/Ensemble/dtcwt_rf.bin','wb') as mod:
    pickle.dump(dtcwt_rf,mod)
# https://scikit-learn.org/stable/model_persistence.html

print(dtcwt_rf.score(X_dtcwt_test_f, y_dtcwt_test_f))
ypred_dtcwt_rf = dtcwt_rf.predict(X_dtcwt_test_f)

0.6974789915966386


Plot a confusion matrix to corroborate the behavior of the model for each genre

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns
from sklearn.metrics import confusion_matrix

# create confusion matrix
cm = confusion_matrix(y_dtcwt_test_f, ypred_dtcwt_rf)

accuracy = np.trace(cm) / np.sum(cm).astype('float')
misclass = 1 - accuracy

cmap = plt.get_cmap('Blues')

# plot confusion matrix
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title('Confusion matrix DTCWT Random Forest')
plt.colorbar()

tick_marks = np.arange(len(GENRES))
plt.xticks(tick_marks, GENRES, rotation=45)
plt.yticks(tick_marks, GENRES)

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, "{:,}".format(cm[i, j]),
            horizontalalignment="center",
            color="white" if cm[i, j] > (cm.max() / 2) else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
plt.savefig('ConfM_DTCWT_rf.png')

A Cross-Validation using K-Fold to validate the accuracy in the previous fit

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
kfold=KFold(n_splits=5 , shuffle=True,random_state=0)
rf_scores = cross_val_score(dtcwt_rf, X_dtcwt, y, cv=kfold)

print('scores: ',rf_scores) # [0.735      0.705      0.8        0.81       0.79396985]
print('mean: ',rf_scores.mean()) # 0.7687939698492463

import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.title('Cross-Validation Scores in Random Forest for DTCWT')
plt.xlabel('Fold')
plt.ylabel('Score')
ax = plt.gca()
ax.set_xlim(0.9, 5.1)
ax.set_ylim(0.6, 1.01)
plt.grid()
plt.plot(range(1,6),rf_scores,'o-',color='blue',lw=2)
plt.plot(range(1,6),[rf_scores.mean()]*5, linestyle="-.",color='k')
plt.annotate("%0.4f" % rf_scores.mean(), (3,rf_scores.mean() + 0.005))
plt.legend(['accuracy','mean acc'],loc="best")
plt.savefig('CV_DTCWT_rf.png')